<h1 align="center">Детекция объектов</h1>

<!-- <h1 style="color:#008B8B">1. Non-maximum supression</h1> -->

## Non-maximum supression

* Модель выдаёт для класса $k$ список прямоугольников с уверенностями
* Проходим в порядке уменьшения уверенности
* Для каждого прямоугольника удаляем все последующие, с которыми Intersection over Union (IoU) > 0.5

Имеется $k$ классов и для каждого класса, модель выдаёт список прямоугольников с уверенностями. Возьмём первый прямоугольный (увверенность 0.9), из тех, которые модель выдала для первого класса. Среди всех следующих прямоугольников данного класса находим те, у которых (IoU) > 0.5. Переходим к следующему прямоугольники и повторяем операцию.

## Метрики качества

* Модель выдаёт для класса $k$ список прямоугольников с уверенностями
* Считаем прямоугольник корректным, если $\text{loU}(y,z) = \text{Jaccard}(y, z) > t$
* Строим PR-кривую, считаем под ней площадь, получаем Average Precision
* Усредняем по всем классам, получаем mAP (mean AP)
* Раньше $t = 0.5$, сейчас скорее $t = 0.75$

Имеется размеченная картинка, модель расставила на данной картинке некоторые прямоугольники. Каждый из прямоугольников модели считаются корректными, если $\text{loU}(y,z) = \text{Jaccard}(y, z) > t$. Благодоря данному порогу мы определяем, правлиьный ли прямоугольник.

Мы получили список прямоугольникоов, которые отсортированы по вероятностям принадлжености к некоторому классу. Дальше, проставляем метки правлиьности используя порог $t$ (loU) и считаем площадь под PR кривой. При построении PR кривой, мы сдвигаем порог, но используется порог на вероятности, которые выдает модель, а не loU.

<h1 style="color:#008B8B">1. Two-shot detection</h1>

Будем решать задачу в два этапа:
1. Находим «кандидатов» — прямоугольники, где скорее всего что-то есть
2. Классифицируем эти прямоугольники

## R-CNN

1. Имеется входная картинка.
2. Применяем, метод, которые формирует прямоугольники на картинке (~2000 штук). Берем каждый прямоугольник, выполняем преобразование его размера.
3. Данный прямоугольник передаём предобученной неройнной сети (AlexNet...), из него достаём признаки с предпоследнего слоя.
4. Используя данное признаковое описание обучаем классификатор (SVM...), который по данным признакм будет определять то, что находится в данном прямоугольнике.

<img src='img/lecture08/1.png'>

**Генерация кандидатов:**

* «Внешние» методы из классического компьютерного зрения
* Выбирается около 2000 прямоугольников

Используем методы классического компьютерного зрения. Берём все пиксели, для каждого пикслея берём его признаки, координаты, цвета... и кластеризуем все пиксели. Один кластер называется суперпикселем. Выбираем прямоугольники так, чтобы они покрывали все спуерпиксели.

<img src='img/lecture08/2.png'>

Классификация прямоугольников:
* SVM
* One-vs-all

### Проблемы:
* Не end-to-end
* Генерация кандидатов может быть очень сложной
* Свёрточная сеть практически не настраивается под данные
* Долго (много признаков, много классификаторов)

## Fast R-CNN

Кандидаты всё ещё генерируются внешним методом. Но 3 и 4 шаг объеденили в один.

<img src='img/lecture08/3.png'>

* Для конкретного кандидата извлекаются признаки: вырезаются участки из последнего тензора, режутся на блоки, из каждого блока берётся максимум.

### Скорость

<img src='img/lecture08/6.png'>

## Faster R-CNN

<img src='img/lecture08/7.png'>

### Скорость

<img src='img/lecture08/9.png'>

<img src='img/lecture08/10.png'>

<h1 style="color:#008B8B">2. One-shot detection</h1>

* Двухэтапные детекторы работают хорошо, но не очень быстро
* Попытаемся одновременно и искать кандидатов, и определять их классы

## YOLO

1. Имеется входная картинка, которая разбита на S*S блоков;

2. Пропускаем через сверточную архитектуру нейронной сети DarkNet (ResNet), берем последний свёрточный слой архитектуры $7 \times 7 \times 1024$;

3. Применяем несколько полносвязынх слоёв, таких, что для каждой ячейки, полносвязный слой возвращает B характеристик прямоугольников (описаны в скобках и ниже) и вероятности конкретных классов при условии, что в данном прямоугольнике, что в прямоугольнике имеется объект;

<img src='img/lecture08/11.png'>


### Идея

* Разбиваем изображение на $S\times S$ блоков;
* Для каждого блока предсказываем $B$ прямоугольников, это число фиксировано. Для каждого прямоугольника имеется:
    * Координаты центра прямоугольника $(x, y)$, ширина и высота этого прямогульника. Таких прямоугольников один блок должен выдать $B$ штук, где центры данных прямоугольников внутри блока, но края могут выходить;
    * Вероятность наличия объекта;
    * Вероятность каждого класса **при условии** наличия объекта - окрашиваем блоки в цвета калссов (картинка ниже);

<img src='img/lecture08/12.png'>

### Качество 

<img src='img/lecture08/13.png'>

<h1 align="center">Идентификация объектов</h1>

<h1 style="color:#008B8B">3. DeepFace</h1>

<img src='img/lecture08/15.png'>

* Обучаем некоторую архитектуру для классификации (число классов = число людей в данных)
* Используем выходы предпоследнего слоя как признаковое описание изображения
* Признаки нормализуются (чтобы норма была единичной)
* Лица похожих людей находятся в близких точках. Если на вход приходит новое лицо, которое пропускаем через нейронную сеть, получаем вектор. Считаем близость векторов по какой-нибудь метрике с лицами из базы, у которых векторы уже получены.


Можно сравнить расстояние с порогом, чтобы идентифицировать человека, так как ближайший вектор может оказаться достаточно далеко. Порог позволяет не относить человека к таким случаем, мы просто не уверены в данной ситуации.

Опять наблюдаем проблему, что задача решается в несколько этапов. Изначально примеются задачи классического компьютерного зрения для выделения лица. После, обучение нейронной сети извлекаем полезные признаки, прменяем метод ближайших соседей или что-то подобное. Хочется сразу обучать нейронную сеть на идентификацию лиц. 

<h1 style="color:#008B8B">3. FaceNet</h1>

Почему бы в явном виде не обучать представления изображений так, чтобы фотографии одного человека имели близкие представления?

Обучаем нейронную сеть на парах изображений. Необходимы фотографии, где на изображении один и тот же человек, и фотография другого человека.

<img src='img/lecture08/16.png'>

Нейронная сеть будет обучаться на триплетах. Один элемент обучающей выборки представляет собой тройку картинок. Первая картинка - это Anchor или же якорь. Positive - это фотография того же человека, Negative - фотография другого человека.

<img src='img/lecture08/17.png'>

Посмотрим на функцию потреь. Береём конкретную тройку, где $x_i^a$ - якорь из тройки, $x_i^p$ - человек того же класса. $x_i^n$ - другой человек. $f(x)$ - это векторное представление. Мы берём вектор якоря, считаем расстояние до того же человека. Дальше, считаем расстояние от якоря до другого человека. Считаем разницу между двумя расстояниями и прибавляем $\alpha$ и выполняем положительную срезку ($z_+ = \max (z, 0)$). Если выражение в скобках меньше нуля, тогда квадратные скобки будут равны нулю, а если выражение больше нуля, тогда квадратные скобки будут равны знечению под ними.

Выражение внутри скобок будет отрицательным, если растояние до другого человка окажется больше, чем расстояние до того же человека + $\alpha$, тогда ошибка будет равна нулю.

Мы считаем Евклидово расстояние, так как мы обучаем пространство эмбеддингов так, чтобы Евклидово расстояние работало хорошо. На этапе применения имеется фотография владельца телефона, и то, что видит камера. Считаем расстояние между двумя фотографиями и если оно меньше некоторго порога, тогда говорим, что это один человек. Триплеты используются только на этапе обучаения, а на этапе применения мы используем признаковое описание из нейронной сети.

### Выбор триплетов

* Важно правильно выбирать триплеты
* Обычно: выбираем positive и ищем semi-hard negatives

$$\large | f(x_i^a) - f(x_i^p)|_2^2 < | f(x_i^a) - f(x_i^n) |_2^2$$

Правильнее всего взять якорь, взять позитивный пример и искать негативные такие, чтобы было выполнено данное условие. 

* Точность на LFW: 99.63%